In [29]:
import asyncio

import keyring
import getpass
import github
import datetime
import requests

In [2]:
# access_token = getpass.getpass()
# keyring.set_password("github-access-token", "radiotherapyai-bot", access_token)

In [31]:
access_token = keyring.get_password("github-access-token", "radiotherapyai-bot")

headers = {"Authorization": f"Bearer {access_token}"}

In [32]:
url = 'http://api.github.com/graphql'

In [41]:
query = """
query($owner: String!, $name: String!, $path: String!) {
  repository(owner: $owner, name: $name) {
      object(expression: "main") {
        ... on Commit {
          blame(path: $path) {
            ranges {
              startingLine
              endingLine
              age
              commit {
                oid
                author {
                  name
                }
              }
            }
          }
        }
      }
  }
  viewer {
    login
  }
  rateLimit {
    limit
    cost
    remaining
    resetAt
  }
}
"""

variables = {
    "path": "src/documents/released/sop-document-record-control/index.md",
    "owner": "RadiotherapyAI",
    "name": "regulatory"
}

In [42]:
request = requests.post('https://api.github.com/graphql', json={'query': query, 'variables': variables}, headers=headers)

In [43]:
request.json()

{'data': {'repository': {'object': {'blame': {'ranges': [{'startingLine': 1,
       'endingLine': 3,
       'age': 3,
       'commit': {'oid': '9fb18c55cbb26ea2357fda9bc4808e530d21497e',
        'author': {'name': 'Simon Biggs'}}},
      {'startingLine': 4,
       'endingLine': 4,
       'age': 3,
       'commit': {'oid': 'be9163ae62bb166d812ffbf3af2a8af3dded30da',
        'author': {'name': 'Simon Biggs'}}},
      {'startingLine': 5,
       'endingLine': 9,
       'age': 3,
       'commit': {'oid': '9fb18c55cbb26ea2357fda9bc4808e530d21497e',
        'author': {'name': 'Simon Biggs'}}},
      {'startingLine': 10,
       'endingLine': 14,
       'age': 3,
       'commit': {'oid': 'be9163ae62bb166d812ffbf3af2a8af3dded30da',
        'author': {'name': 'Simon Biggs'}}},
      {'startingLine': 15,
       'endingLine': 17,
       'age': 3,
       'commit': {'oid': '9fb18c55cbb26ea2357fda9bc4808e530d21497e',
        'author': {'name': 'Simon Biggs'}}},
      {'startingLine': 18,
       'endin

In [5]:
github_instance

In [6]:
for repo in github_instance.get_user().get_repos():
    print(repo.name)

regulatory


In [7]:
repo = github_instance.get_repo("RadiotherapyAI/regulatory")

In [8]:
repo.name

'regulatory'

In [9]:
pull_requests = list(repo.get_pulls(direction='asc'))

In [10]:
pr_with_an_approval = repo.get_pull(20)

In [11]:
reviews = list(pr_with_an_approval.get_reviews())

In [12]:
for review in reviews:
    if review.state == "APPROVED":
        print(f"{review.user.name} approved {review.commit_id} on the {review.submitted_at.strftime('%Y-%b-%d')}")

Oliver Eidel approved ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5 on the 2022-Mar-01


In [13]:
review = reviews[0]

In [14]:
review.commit_id

'ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5'

In [15]:
review.user

NamedUser(login="olieidel")

In [16]:
review.submitted_at.strftime('%Y-%b-%d')

'2022-Mar-01'

In [17]:
list()

[]

In [18]:
for a_file in pr_with_an_approval.get_files():    
    print(f"https://github.com/RadiotherapyAI/regulatory/blob/{review.commit_id}/{a_file.filename}")

https://github.com/RadiotherapyAI/regulatory/blob/ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5/src/documents/drafts/62304-mapping.md
https://github.com/RadiotherapyAI/regulatory/blob/ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5/src/documents/drafts/quality-manual-policy-objectives.md
https://github.com/RadiotherapyAI/regulatory/blob/ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5/src/documents/drafts/sop-integrated-software-development.md


In [21]:
pr_with_an_approval.merged_by.name

'Simon Biggs'

In [20]:
pr_with_an_approval.merged_at

datetime.datetime(2022, 3, 23, 5, 57, 58)

In [57]:
commits = list(repo.get_commits(path='src/documents/released/sop-document-record-control/index.md'))

In [58]:
commit = commits[0]

In [59]:
commit.committer.name

'GitHub Web Flow'

In [60]:
list(commit.get_statuses())

[]

In [61]:
commit.last_modified

'Fri, 04 Mar 2022 23:39:36 GMT'

In [62]:
def commit_date(commit):
    return datetime.datetime.strptime(commit.last_modified, "%a, %d %b %Y %H:%M:%S %Z")
    

In [63]:
commit_date(commit).strftime('%Y-%b-%d')

'2022-Mar-04'

In [64]:
author_and_timestamp = []
prs_for_this_file = set()

for commit in commits:
    author_and_timestamp.append((commit.committer.name, commit_date(commit)))
    prs_for_this_file.update(commit.get_pulls())
    
prs_for_this_file

{PullRequest(title="Allow for PR only edits", number=18),
 PullRequest(title="Release sop drc, remove rdm templates for now", number=14)}

In [ ]:
# Doesn't have the full history of the file.
# Potentially need to use the GraphQL API?

In [65]:
author_and_timestamp = sorted(author_and_timestamp, key=lambda x: x[1])
author_and_timestamp

[('GitHub Web Flow', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('GitHub Web Flow', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('GitHub Web Flow', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36))]

In [66]:
file_creator = author_and_timestamp[0][0]
file_creator

'GitHub Web Flow'

In [67]:
prs_for_this_file = list(commit.get_pulls())
prs_for_this_file

[PullRequest(title="Release sop drc, remove rdm templates for now", number=14)]